2022-05-17 <br/>
Eurovision <br/>
https://github.com/rfordatascience/tidytuesday/blob/master/data/2022/2022-05-17/readme.md

Line Maps - Completed  <br/>
Animated line Maps - Incomplete

Special thanks to: JP Hwang <br/>
https://medium.com/swlh/interactive-animated-travel-data-visualizations-mapping-nba-travel-a154a2e7c18d

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2022/2022-05-17/eurovision-votes.csv')

In [4]:
df.head()

,year,semi_final,edition,jury_or_televoting,from_country,to_country,points,duplicate
0,1975,f,1975f,J,Belgium,Belgium,0,x
1,1975,f,1975f,J,Belgium,Finland,0,NaN
2,1975,f,1975f,J,Belgium,France,2,NaN
3,1975,f,1975f,J,Belgium,Germany,0,NaN
4,1975,f,1975f,J,Belgium,Ireland,12,NaN


In [5]:
df.shape

(56312, 8)

In [6]:
df.duplicate.value_counts()

x    1530
Name: duplicate, dtype: int64

In [11]:
# remove duplicates
df_reduced= df[(df.duplicate.isna()) & (df.year>=2000)]

In [12]:
df_reduced

,year,semi_final,edition,jury_or_televoting,from_country,to_country,points,duplicate
11371,2000,f,2000f,J,Austria,Belgium,0,NaN
11372,2000,f,2000f,J,Austria,Croatia,6,NaN
11373,2000,f,2000f,J,Austria,Cyprus,0,NaN
11374,2000,f,2000f,J,Austria,Denmark,10,NaN
11375,2000,f,2000f,J,Austria,Estonia,3,NaN
...,...,...,...,...,...,...,...,...
56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN
56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN
56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN
56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN


In [13]:
# 0 if same country, -1 if different
df_reduced['vote_region']= df_reduced.apply(lambda x: 1 if x.from_country!=x.to_country else -1 , axis=1)

C:\Users\shawn\AppData\Local\Temp\ipykernel_15104\2714545491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['vote_region']= df_reduced.apply(lambda x: 1 if x.from_country!=x.to_country else -1 , axis=1)


In [14]:
df_reduced

,year,semi_final,edition,jury_or_televoting,from_country,to_country,points,duplicate,vote_region
11371,2000,f,2000f,J,Austria,Belgium,0,NaN,1
11372,2000,f,2000f,J,Austria,Croatia,6,NaN,1
11373,2000,f,2000f,J,Austria,Cyprus,0,NaN,1
11374,2000,f,2000f,J,Austria,Denmark,10,NaN,1
11375,2000,f,2000f,J,Austria,Estonia,3,NaN,1
...,...,...,...,...,...,...,...,...,...
56307,2022,f,2022f,T,United Kingdom,Serbia,1,NaN,1
56308,2022,f,2022f,T,United Kingdom,Spain,5,NaN,1
56309,2022,f,2022f,T,United Kingdom,Sweden,4,NaN,1
56310,2022,f,2022f,T,United Kingdom,Switzerland,0,NaN,1


A few ways of going with this <br/>
* By number of votes <br/>
* By sum of votes <br/>
* By Weighted average of votes   <--- 

In [18]:
def weighted_average(df, values, weights):
    return round(sum(df[weights] * df[values]) / df[weights].sum(),2)


df_weighted= df_reduced.groupby(['year','from_country','to_country']).apply(weighted_average, 'points', 'vote_region').reset_index(name='weighted_avg')

In [69]:
# Pulling the global coordinates 
# Special thanks to albertyw   : https://github.com/albertyw/avenews/blob/master/old/data/average-latitude-longitude-countries.csv
df_country = pd.read_csv('https://raw.githubusercontent.com/albertyw/avenews/master/old/data/average-latitude-longitude-countries.csv')

In [70]:
df_country.rename(columns={"ISO 3166 Country Code": "Country Code"},inplace=True)

In [71]:
df_from_country_code =  df_country.add_suffix('_from_country_code')
df_to_country_code=  df_country.add_suffix('_to_country_code')

In [72]:
df_from_country_code.head()

,Country Code_from_country_code,Country_from_country_code,Latitude_from_country_code,Longitude_from_country_code
0,AD,Andorra,42.50,1.50
1,AE,United Arab Emirates,24.00,54.00
2,AF,Afghanistan,33.00,65.00
3,AG,Antigua and Barbuda,17.05,-61.80
4,AI,Anguilla,18.25,-63.17


In [106]:
# Merging everything together
full_df= df_weighted.merge(df_from_country_code,how='inner',left_on='from_country', right_on='Country_from_country_code')
full_df= full_df.merge(df_to_country_code,how='inner',left_on='to_country', right_on='Country_to_country_code')

In [107]:
full_df.head()

,year,from_country,to_country,weighted_avg,Country Code_from_country_code,Country_from_country_code,Latitude_from_country_code,Longitude_from_country_code,Country Code_to_country_code,Country_to_country_code,Latitude_to_country_code,Longitude_to_country_code
0,2000,Austria,Belgium,0.0,AT,Austria,47.33,13.33,BE,Belgium,50.83,4.0
1,2002,Austria,Belgium,0.0,AT,Austria,47.33,13.33,BE,Belgium,50.83,4.0
2,2003,Austria,Belgium,4.0,AT,Austria,47.33,13.33,BE,Belgium,50.83,4.0
3,2004,Austria,Belgium,0.0,AT,Austria,47.33,13.33,BE,Belgium,50.83,4.0
4,2005,Austria,Belgium,0.0,AT,Austria,47.33,13.33,BE,Belgium,50.83,4.0


In [35]:
full_df= full_df[full_df.year==2022].reset_index()

In [36]:
full_df

,index,year,from_country,to_country,weighted_avg,Country Code_from_country_code,Country_from_country_code,Latitude_from_country_code,Longitude_from_country_code,Country Code_to_country_code,Country_to_country_code,Latitude_to_country_code,Longitude_to_country_code
0,14,2022,Austria,Belgium,0.00,AT,Austria,47.33,13.33,BE,Belgium,50.83,4.00
1,28,2022,Croatia,Belgium,0.00,HR,Croatia,45.17,15.50,BE,Belgium,50.83,4.00
2,46,2022,Cyprus,Belgium,2.75,CY,Cyprus,35.00,33.00,BE,Belgium,50.83,4.00
3,62,2022,Denmark,Belgium,0.00,DK,Denmark,56.00,10.00,BE,Belgium,50.83,4.00
4,81,2022,Estonia,Belgium,3.00,EE,Estonia,59.00,26.00,BE,Belgium,50.83,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,20868,2022,Albania,Austria,2.00,AL,Albania,41.00,20.00,AT,Austria,47.33,13.33
1136,20891,2022,Bulgaria,Austria,0.00,BG,Bulgaria,43.00,25.00,AT,Austria,47.33,13.33
1137,20909,2022,Armenia,Austria,2.50,AM,Armenia,40.00,45.00,AT,Austria,47.33,13.33
1138,20966,2022,Italy,Austria,2.50,IT,Italy,42.83,12.83,AT,Austria,47.33,13.33


In [63]:
df_country.rename(columns={"ISO 3166 Country Code": "Country Code"},inplace=True)

In [67]:
df_sending_country_code =  df_country.add_suffix('_from_country_code')
df_receiving_country_code=  df_country.add_suffix('_to_country_code')

In [ ]:
# Merging everything together
full_df= df_2019.merge(df_sending_country_code,how='inner',left_on='sending_country_code', right_on='Country Code_sending_country_code')
full_df= full_df.merge(df_receiving_country_code,how='inner',left_on='receiving_country_code', right_on='Country Code_receiving_country_code')
full_df= full_df.merge(df_counts,how='inner',left_on='participant_nationality', right_on='participant_nationality')

In [138]:
# With all users - With a gradient scale 
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

# fig.add_trace(go.Scattergeo(
#     locationmode = 'country names' ,
#     hoverinfo = 'text',
#     text = full_df['company_location'],
#     lon=full_df['Longitude_bean_origin'],
#     lat=full_df['Latitude_bean_origin'],
#     mode = 'markers',
#     marker = dict(
#         size = 2,
#         color = 'rgb(255, 0, 0)',
#         line = dict(
#             width = 3,
#             color = 'rgba(68, 68, 68, 0)'
#         )
#     )))

for i in range(len(full_df)):
    fig.add_trace(
        go.Scattergeo(
          #  locations=full_df['company_location'],
            hoverinfo = 'text',
            text = f'{full_df.from_country[i]} voted for {full_df.to_country[i]} averaging {full_df.weighted_avg[i]} votes',
            locationmode = 'country names' ,
            lon = [full_df['Longitude_from_country_code'][i], full_df['Longitude_to_country_code'][i]],
            lat = [full_df['Latitude_from_country_code'][i], full_df['Latitude_to_country_code'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(full_df['weighted_avg'][i])/100
        )
    )

fig.update_layout(
    title_text = "Who voted and for whom in Eurovison 2022?",
    showlegend = False,
    geo = dict(
        showland = True,
        #scope = 'europe',
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

In [140]:
# ========== ANIMATE TRAVEL PATHS ==========
# FOR ONE TEAM

import numpy as np

teamname = 'Ukraine'
# travel_team_df = travel_df[travel_df.teamname == teamname]
team_col = (full_df['weighted_avg']/10).apply(float)

countries = full_df.from_country.unique().tolist()
lon_vals = np.append(full_df['Longitude_from_country_code'].values, full_df['Longitude_to_country_code'].values[-1])
lat_vals = np.append(full_df['Latitude_from_country_code'].values, full_df['Latitude_to_country_code'].values[-1])

frames = list()
lon_data = np.array(lon_vals[0])
lat_data = np.array(lat_vals[0])

for i in range(len(lon_vals)):
    frames.append(
        go.Frame(data=[go.Scattergeo(lon=lon_data, lat=lat_data)])
    )
    lon_data = np.append(lon_data, lon_vals[i])
    lat_data = np.append(lat_data, lat_vals[i])

fig = go.Figure(
    data=[
        go.Scattergeo(
            locationmode='country names'
            , mode="lines",
            lon=np.append(lon_vals[0], lon_vals[0]),
            lat=np.append(lat_vals[0], lat_vals[0]),
            line=dict(width=1, color = 'blue')
            #, color=team_col)
            , opacity=team_col[0],
            hoverinfo='text',
            text='AAA',
            #name=countries
        )
    ],
    layout=go.Layout(
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play", method="animate", args=[None])])]
    ),
    frames=frames
)


# fig.add_trace(go.Scattergeo(
#     lon=full_df['Longitude_from_country_code']
#     , lat=full_df['Latitude_from_country_code']
#     , marker=dict(size=8, color='slategray')
#     # , hoverifo='text'
#     # , text=f'{full_df.from_country[i]} voted for {full_df.to_country[i]} averaging {full_df.weighted_avg[i]} votes'
#     , name='Arenas'
# ))

for i in range(len(full_df)):
    fig.add_trace(
        go.Scattergeo(
          #  locations=full_df['company_location'],
            hoverinfo = 'text',
            text = f'{full_df.from_country[i]} voted for {full_df.to_country[i]} averaging {full_df.weighted_avg[i]} votes',
            locationmode = 'country names' ,
            lon = [full_df['Longitude_from_country_code'][i], full_df['Longitude_to_country_code'][i]],
            lat = [full_df['Latitude_from_country_code'][i], full_df['Latitude_to_country_code'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = float(full_df['weighted_avg'][i])/100
        )
    )


fig['data'][1]['marker']['symbol'] = 'triangle-left'
fig.update_layout(
    title_text='Who voted for Ukraine in 2022',
    geo=dict(
        #scope='north america',
       # projection_type='azimuthal equal area',
        showland=True,
        fitbounds="locations",
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
    ),
)
fig.show()

In [ ]:
# ========== ANIMATE TRAVEL PATHS ==========
# FOR ONE TEAM
teamname = 'DALLAS_MAVERICKS'
travel_team_df = travel_df[travel_df.teamname == teamname]
team_col = teamcolor_dict[teamname]

lon_vals = np.append(travel_team_df['orig_lon'].values, travel_team_df['dest_lon'].values[-1])
lat_vals = np.append(travel_team_df['orig_lat'].values, travel_team_df['dest_lat'].values[-1])

frames = list()
lon_data = np.array(lon_vals[0])
lat_data = np.array(lat_vals[0])

for i in range(len(lon_vals)):
    frames.append(
        go.Frame(data=[go.Scattergeo(lon=lon_data, lat=lat_data)])
    )
    lon_data = np.append(lon_data, lon_vals[i])
    lat_data = np.append(lat_data, lat_vals[i])

fig = go.Figure(
    data=[
        go.Scattergeo(
            locationmode='USA-states', mode="lines",
            lon=np.append(lon_vals[0], lon_vals[0]),
            lat=np.append(lat_vals[0], lat_vals[0]),
            line=dict(width=1, color=team_col), opacity=0.1,
            hoverinfo='none', name=teamname
        )
    ],
    layout=go.Layout(
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play", method="animate", args=[None])])]
    ),
    frames=frames
)
fig.add_trace(go.Scattergeo(
    lon=arena_df['lon'], lat=arena_df['lat'], marker=dict(size=8, color='slategray'),
    hoverinfo='text', text=arena_df['teamname'] + ' - ' + arena_df['arena_name'], name='Arenas'
))
fig['data'][1]['marker']['symbol'] = 'triangle-left'
fig.update_layout(
    title_text='NBA Travel paths',
    geo=dict(
        scope='north america',
        projection_type='azimuthal equal area',
        showland=True,
        fitbounds="locations",
        landcolor='rgb(243, 243, 243)',
        countrycolor='rgb(204, 204, 204)',
    ),
)
fig.show()